## Step 1: Import Required Libraries

We import `pandas` for data handling, `PuLP` for building the linear programming model, and `matplotlib` for visualizing the results.


In [12]:
import pandas as pd
from pulp import LpProblem, LpMaximize, LpVariable, lpSum, LpStatus, value




In [13]:
df = pd.read_csv("data4.csv")  # Make sure the file is in the same folder as your notebook
df.head()  # This will show the first few rows of the CSV file


,Product,Profit,Material,Labor,MaxProduction
0,A,30,5,6,100
1,B,20,4,3,80
2,C,40,7,5,60


## Step 2: Load Dataset

We load the dataset `data4.csv`, which contains product names and resource usage.


In [14]:
model = LpProblem("Product Mix Optimization", LpMaximize)
products = df['Product'].tolist()
decision_vars = LpVariable.dicts("Produce", products, lowBound=0, cat='Integer')


c:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\pulp\pulp.py:1455: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


## Step 3: Define the Optimization Problem

We define a maximization problem using PuLP. This sets up the structure of the model we want to solve.


In [15]:
# Objective
model += lpSum(decision_vars[p] * df.loc[df['Product'] == p, 'Profit'].values[0] for p in products), "Total_Profit"

# Material Constraint
model += lpSum(decision_vars[p] * df.loc[df['Product'] == p, 'Material'].values[0] for p in products) <= 1000, "Material_Constraint"

# Labor Constraint
model += lpSum(decision_vars[p] * df.loc[df['Product'] == p, 'Labor'].values[0] for p in products) <= 1200, "Labor_Constraint"

# Max Production
for p in products:
    model += decision_vars[p] <= df.loc[df['Product'] == p, 'MaxProduction'].values[0], f"MaxProduction_{p}"


## Step 4: Add Objective Function and Constraints

We add:
- The **objective function** to maximize total profit.
- **Constraints** to limit total usage of material and labor based on availability.


In [16]:
model.solve()


1

## Step 5: Solve the Optimization Model

We use the built-in `.solve()` function to find the optimal solution for our decision variables.


In [17]:
print("\n✅ Optimization Result:")
print(f"Status: {LpStatus[model.status]}")
for p in products:
    print(f"{p}: {decision_vars[p].varValue}")
print(f"Total Profit: {value(model.objective)}")



✅ Optimization Result:
Status: Optimal
A: 100.0
B: 20.0
C: 60.0
Total Profit: 5800.0


## Step 6: Display the Optimization Results

We print the status of the solution, the number of units to produce for each product, and the total profit obtained.


quantities = [decision_vars[p].varValue for p in products]
plt.bar(products, quantities, color='skyblue')
plt.title("Optimal Production Plan")
plt.xlabel("Product")
plt.ylabel("Units Produced")
plt.grid(axis='y')
plt.tight_layout()
plt.savefig("optimal_production_plan.png")
plt.show()


## Step 7: Plot the Optimal Production Plan

We create a bar chart showing the number of units to produce for each product. The chart is saved as an image file.


In [18]:
print("\n🔍 Insights:")
print(". The model produced 100 units of Product A, 20 of B, and 60 of C.")
print(". Product A and C are likely more profitable or resource-efficient.")
print(". Product B is produced in low quantity, possibly due to higher resource use or lower profit.")
print(". Total profit is 5800 units under the given constraints (Material: 1000, Labor: 1200).")
print(". Resources are optimally allocated, and no constraint is violated.")



🔍 Insights:
. The model produced 100 units of Product A, 20 of B, and 60 of C.
. Product A and C are likely more profitable or resource-efficient.
. Product B is produced in low quantity, possibly due to higher resource use or lower profit.
. Total profit is 5800 units under the given constraints (Material: 1000, Labor: 1200).
. Resources are optimally allocated, and no constraint is violated.


## Step 8: Business Insights

Based on the optimization results, we can draw the following business conclusions:

- ✅ **Product A** and **Product C** are the most profitable or resource-efficient, as the model chose to produce **100 units of A** and **60 units of C**.
- ⚠️ **Product B** is produced in **low quantity (20 units)**, suggesting it either has lower profit or consumes more resources compared to A and C.
- 💰 The **total profit is 5800**, showing the maximum gain achievable under the given constraints.
- 🔧 **Resource constraints** (Material = 1000, Labor = 1200) are fully utilized, meaning resources were **optimally allocated**.
- 📈 The model ensures no constraint is violated, and all production quantities are within the maximum allowed limits.
- 🧠 This insight helps management prioritize **Product A and C** for future planning, and possibly reconsider the role of Product B in the mix.
